In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import openpyxl as op
import time
import threading


# Initialize the WebDriver
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ['enable-automation'])
options.add_argument('--disable-blink-features')   
options.add_argument('--disable-blink-features=AutomationControlled')   
options.add_argument('--disable-gpu')  

driver = webdriver.Chrome(options=options)
driver.maximize_window()
wait = WebDriverWait(driver, 10)

# Comment scrape function
def scrape_comments(product_url):
    driver.get(product_url)
    time.sleep(5)  
    if input('请手动登录，确认界面加载完毕，输入数字“1”开始爬取-->') == 1:
        pass
    # Complete Login by scanning QR code and wait for the page to load
    
    # Navigate to the comments section
    comments_tab = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.ShowButton--o4XEG7ih')))
    comments_tab.click()
    time.sleep(5)
    
    if input('确认界面加载完毕，输入数字“1”开始爬取-->') == 1:
        pass
     # Wait for the comments to load
    comments_data = []
    comments = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.Comment--KkPcz74T')))
    
       
    for comment in comments:
       user = comment.find_element(By.CSS_SELECTOR, '.userInfo--ccnYGw2O .userName--mmxkxkd0').text
       content = comment.find_element(By.CSS_SELECTOR, '.content--FpIOzHeP').text
       date = comment.find_element(By.CSS_SELECTOR, '.meta--TDfRej2n').text
       
           
       comments_data.append({
           'User': user,
           'Content': content,
           'Date and style': date
           })
    return comments_data

    
    
        
    
# Save data to Excel 
def save_to_excel(comments_data, filename):
    wb = op.Workbook()
    ws = wb.active
    ws.title = "Comments"

    # Create header
    headers = ['User', 'Content', 'Date and style']
    for col_num, header in enumerate(headers, 1):
        ws.cell(row=1, column=col_num, value=header)

    # Write data
    for row_num, comment in enumerate(comments_data, 2):
        ws.cell(row=row_num, column=1, value=comment['User'])
        ws.cell(row=row_num, column=2, value=comment['Content'])
        ws.cell(row=row_num, column=3, value=comment['Date and style'])
       

    wb.save(filename)
    print(f"Data saved to {filename}")

# Input products URL and Excel file name
if __name__ == '__main__':
    product_url = input("Enter the product URL: ")
    product_url = 'https:' + product_url
    filename = input("Enter the filename to save the comments: ")
    Filename = filename + '(评论).xlsx'
    
    comments_data = scrape_comments(product_url)
    save_to_excel(comments_data, Filename)

    driver.quit